## Fashion MNIST Image Classification - Multi-GPU training

**Code tested on:**

- Tensorflow==2.1.0
- Tensorflow-datasets==2.1.0


**Key activities**

- Extract and process Fashion-MNIST data
- Build Tensorflow keras model 
- Training on Multiple GPU using MirroredStrategy 
- Evaluate model 



In [1]:
!pip3 install tensorflow-datasets==2.1.0 --user

You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# restart kernel
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

restartkernel() 

### Import libraries

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
tfds.disable_progress_bar()
import logging
from datetime import datetime
logger = tf.get_logger()
logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(message)s",
    datefmt="%Y-%m-%dT%H:%M:%SZ",
    level=logging.INFO)
print('Tensorflow-version: {0}'.format(tf.__version__))

In [ ]:
# clear the logs
!rm -rf logs/

### Data extraction & processing 

In [ ]:
# prepare data
def prepare_data(batch_size=64, shuffle_size=1000):

    def scale(image, label):
        image = tf.cast(image, tf.float32)
        image /= 255
        return image, label
    
    # Split the training set into 80% and 20% for training and validation
    train_validation_split = tfds.Split.TRAIN.subsplit([8, 2])
    ((train_data, validation_data), test_data),info = tfds.load(name="fashion_mnist:1.0.0", 
                                                         split=(train_validation_split, tfds.Split.TEST),
                                                         as_supervised=True, with_info=True)

    
    print("Training data count : ", int(info.splits['train'].num_examples * 0.8))
    print("Validation data count : ", int(info.splits['train'].num_examples * 0.2))
    print("Test data count : ", int(info.splits['test'].num_examples))


    # create dataset to be used for training process
    train_dataset = train_data.map(scale).shuffle(shuffle_size).batch(batch_size).repeat().prefetch(tf.data.experimental.AUTOTUNE)
    val_dataset = validation_data.map(scale).batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    test_dataset = test_data.map(scale).batch(batch_size)
    
    return train_dataset, val_dataset, test_dataset

### Build Model 

In [ ]:
def build_model(learning_rate=0.001):
    # define model architecture
    model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(28, 28, 1), name='x'),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
      ])
    # compile model with loss, optimizer and accuracy 
    model.compile(
        loss=tf.keras.losses.sparse_categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        metrics=['accuracy'])
    return model

### Model Callback 

In [ ]:
def get_callbacks():
    # callbacks 
    # folder to store current training logs
    logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    class customLog(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            logging.info('epoch: {}'.format(epoch + 1))
            logging.info('loss={}'.format(logs['loss']))
            logging.info('accuracy={}'.format(logs['accuracy']))
            logging.info('val_accuracy={}'.format(logs['val_accuracy']))
    callbacks = [
        tf.keras.callbacks.TensorBoard(logdir),
        customLog()
    ]
    return callbacks

### Multi-GPU Training

In [ ]:
# list physical devices available
tf.config.list_physical_devices('GPU')

In [ ]:
# using MirroredStrategy
NUM_GPUS = 2
strategy = tf.distribute.MirroredStrategy(cross_device_ops=tf.distribute.HierarchicalCopyAllReduce())
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
with strategy.scope():
    # Data extraction and processing
    # set variables
    BUFFER_SIZE = 10000
    BATCH_SIZE = 64 * strategy.num_replicas_in_sync

    train_dataset, val_dataset, test_dataset = prepare_data(batch_size=BATCH_SIZE, shuffle_size=BUFFER_SIZE)
    
    TF_LEARNING_RATE = 0.001
    # build model
    model = build_model(learning_rate=TF_LEARNING_RATE)
    model.summary()
    # train model
    TF_EPOCHS=20
    TF_STEPS_PER_EPOCHS = int(np.ceil(60000 / float(BATCH_SIZE)))  

    model.fit(train_dataset, 
              epochs=TF_EPOCHS,
              steps_per_epoch=3,
              validation_data=val_dataset,
              callbacks=get_callbacks())

**Track GPU Usage** 

If you want to track the GPU usage then, open a terminal and use `nvidia-smi` command. To get refreshed value you can use the `watch -n <NUM_SECONDS>` command. 

`watch -n 1 nvidia-smi`


In [ ]:
# evaluate model
result = model.evaluate(test_dataset, steps=1)
loss = result[0]
accuracy = result[1]
print("loss : {0} accuracy : {1}".format(loss, accuracy))

#### Tensorboard
Note : If you want to use Tensorboard : use tensorboard command 

```
tensorboard --logdir=/home/jovyan/logs/ --bind_all
```
if you are running inside a **container** you can use **port-mapping**. if you are running inside **kubernetes pod**, then use the pod **port-forward feature** on the port 6006 (default for tensorboard, change it as per the tensorboard command output ). When a notebook is created, a pod with name <NOTEBOOK_NAME>-0 is created in the users namespace. So you can use the port-forward to access tensorboard. 

```
kubectl port-forward -n <NAMESPACE> <NOTEBOOK_NAME>-0 6006:6006
```
